In [1]:
# 3:58 form work

In [1]:
# 14Oct at 3:45 test


































``# Supervised Capstone

## Here we will use Houston, Texas, real estate listing data showing which properties are active for sale and which are sold or terminated. We try here to predict those terminated ones. Prior to the prediction, we will perform many cleaning steps and some feature engineering. Later we will use regression models to predict the listing price. This prediction will help to determine if a listing is over- or underpriced. Susch deviation from a correct market price may later explain why the listing get sold or terminated. And to predict this deviation we will use classification methods to see if we can predict an active listing will get terminated.

In [2]:
import matplotlib.pyplot as plt
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from matplotlib.patches import Patch
import matplotlib.patches as patches


import math
import numpy as np
from numpy import percentile
from numpy.random import seed
import os
#from path import Path
import datetime
import time
import re

import pandas as pd
from sqlalchemy import create_engine
from scipy.stats import bartlett
from scipy.stats import boxcox
from scipy.stats import jarque_bera
from scipy.stats import levene
from scipy.stats import normaltest
import scipy.stats as stats
from scipy.stats.mstats import winsorize
from scipy.stats import zscore
import seaborn as sns

In [3]:
import sklearn.datasets
from sklearn import decomposition
from sklearn.datasets import make_blobs
from sklearn.decomposition import PCA
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, fbeta_score, classification_report
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error
import sklearn.metrics as metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet, ElasticNetCV, Lasso, LassoCV, RidgeCV, Ridge
from sklearn.metrics import roc_curve, precision_recall_curve, roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
#from sklearn.model_selection import 
from sklearn.svm import SVC
from sklearn.utils.multiclass import unique_labels

import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse, rmse
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

In [4]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100) 
pd.set_option('display.max_colwidth', -1) 

# Helper Functions 

In [5]:
############################################# DO NOT DELETE ############################################# 

# Help functions to gather basic descriptions
def describe(df):
    return pd.concat([df.describe().T,
                      df.mad().rename('mean abs dev'),
                      df.skew().rename('skew'),
                      df.kurt().rename('kurt'),
                      df.nunique().rename('unique')
                     ], axis=1).T

def data_type_summary(df):
    # get Object data type summary
    df_stat_object = pd.DataFrame([])
    df_stat_object = df.describe(include = ['O'])
    df_stat_object.loc['dtype'] = df.dtypes
    df_stat_object.loc['size'] = len(df)
    #df_stat_object.loc['% null'] = df.isnull().count().round(2)
    #df_stat_object.loc['% null'] = ((df0.isnull().sum()/df0.shape[0])*100).round(3)
    
    
    # get numerical data type summary
    df_stat_num = pd.DataFrame([])
    df_stat_num = df.describe(include = [np.number])
    df_stat_num.loc['dtype'] = df.dtypes
    df_stat_num.loc['size'] = len(df)
    #df_stat_object.loc['% null'] = df.isnull().count().round(2)
    df_stat_object.loc['% null'] = ((df0.isnull().sum()/df0.shape[0])*100).round(3)
    
    # get date data type summary
    df_stat_date = df.describe(include = ['datetime64']) 
    df_stat_date.loc['dtype'] = df.dtypes
    df_stat_date.loc['size'] = len(df)
    #df_stat_object.loc['% null'] = df.isnull().count().round(2)
    #df_stat_object.loc['% null'] = ((df0.isnull().sum()/df0.shape[0])*100).round(3)
    
    # Combine all
    result = pd.concat([df_stat_num, df_stat_object, df_stat_date ], axis=1, sort=False)
    return result

In [6]:
############################################# DO NOT DELETE ############################################# 

def describe_with_nulls(df):
    nan_cols = [i for i in df.columns if df[i].isnull().any()]
    # get Object data type summary
    df_stat_num = pd.DataFrame([])
    df_stat_num = df0[nan_cols].describe(include = [np.number, 'O', 'datetime64']) 
    df_stat_num.loc['dtype'] = df.dtypes
    df_stat_num.loc['size'] = len(df)
    #df_stat_num.loc['% null'] = df.isnull().count().round(2)
    df_stat_num.loc['% null'] = ((df0.isnull().sum()/df0.shape[0])*100).round(3)
    return df_stat_num

In [7]:
############################################# DO NOT DELETE ############################################# 
# Function to produce statistics from scikit learn regression estimators
def regression_results(y_true, y_pred):

    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred) 
    mse=metrics.mean_squared_error(y_true, y_pred) 
    mean_squared_log_error=metrics.mean_squared_log_error(y_true, y_pred)
    median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)

    print('explained_variance: ', round(explained_variance,4))    
    print('mean_squared_log_error: ', round(mean_squared_log_error,4))
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))

In [8]:
############################################# DO NOT DELETE ############################################# 
# Function to move specific column to the left side for easier view
def move_to_left(df, column_name):
    df= df[ [str(column_name)] + [ col for col in df.columns if col != str(column_name) ] ]
    return df

def move_to_left_get_dup(df, column_name):
    df = df[ [str(column_name)] + [ col for col in df.columns if col != str(column_name) ] ]
    df = df[df.duplicated(str(column_name))]
    return df

# Raw data

In [9]:
df_import_cols = ['MLS Number', 'Property Type', 'Status', 'Street Number', 'Street Name', 'Unit Number', 'City/Location' \
                  , 'Zip Code', 'County', 'List Price', 'Close Price', 'Close Date', 'Area', 'Subdivision'\
                  , 'Master Planned Community', 'Market Area', 'School District', 'School Elementary', 'School Middle' \
                  , 'School High', 'Building SqFt','Price Sq Ft List', 'Price Sq Ft Sold', 'Lot Size', 'Acres' \
                  , 'Price Acre List', 'Price Acre Sales', 'Year Built', 'Bedrooms','Baths Full', 'Baths Half' \
                  , 'Baths Total', 'Room Count', 'Fireplaces Number', 'Stories', 'New Construction', 'New Construction Desc' \
                  , 'Pool Private', 'No Of Garage Cap', 'Style', 'DOM', 'CDOM', 'List Office MLSID', 'List Office Name' \
                  , 'List Agent MLSID', 'List Agent Full Name', 'Selling Office MLSID', 'Selling Office Name', 'Selling Agent MLSID' \
                  , 'Selling Agent Full Name', 'Realist Owner Name', 'Last Change Timestamp', 'List Date ']

In [31]:
os.listdir()

['.git',
 'readme.md',
 'Supervised_Capstone.ipynb',
 'Supervised_Capstone1.ipynb']

In [29]:
#path1 = r'C:\Users\User\OneDrive - Seedergy\MAEN\Code\Analytics\\'
#path1 = 'G:\Amir\Analytics\\'
path1 = "C:\Analytics\\"
path2 = 'HAR Pipeline\Step0_Raw\Step0_Data\\'
files = os.listdir(path1+path2)

df_import = pd.DataFrame()
for f in files:
    data = pd.read_excel(path1+f, sheet_name=0, header= None, skiprows=1, names=df_import_cols)
    df_import = df_import.append(data)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Analytics\\HAR Pipeline\\Step0_Raw\\Step0_Data\\'

In [ ]:
df0 = df_import.copy()
# Add a unique ID for future reference
df0['import_id'] = [i for i in range(len(df0))]

In [ ]:
df0.head(2)

In [ ]:
df0.columns

In [ ]:
# Cleanup columns headings
df0.columns = df0.columns.str.lstrip()
df0.columns = df0.columns.str.rstrip()
df0.columns = df0.columns.str.replace(' ', '_')
df0.columns = df0.columns.str.lower()
# Make the whole dataframe to lower case
df0 = df0.applymap(lambda s:s.lower() if type(s) == str else s)

In [ ]:

# Extract the numerical values from string
df0['school_district_num'] = df0['school_district'].str.extract('(\d+)').astype(str)
df0['school_district_num'] = df0['school_district_num'].astype(str).astype(float) 
# Change type to date time
df0["last_change_timestamp"]  = pd.to_datetime(df0["last_change_timestamp"])
df0["list_date"]  = pd.to_datetime(df0["list_date"])
df0["close_date"]  = pd.to_datetime(df0["close_date"])
df0["year_built"]  = pd.to_datetime(df0["year_built"])
# Change currency type to float
currency_cols = ['list_price', 'close_price']
df0[currency_cols] = (df0[currency_cols].replace( '[\$,)]','', regex=True ).replace( '[(]','-',   regex=True ).astype(float))


In [ ]:
# Get a new coutnt of shape
print('The shapes of dataset as we proceed: {}, {}'.format(df_import.shape, df0.shape))

In [ ]:
# Drop columns with many missing values
df0 = df0.drop(['unit_number','close_date', 'master_planned_community','new_construction_desc','realist_owner_name','mls_number' ], axis=1)
# Drop calculated values as these can be recalcualted when we fill the missing values
df0 = df0.drop(['fireplaces_number', 'price_sq_ft_list', 'price_sq_ft_sold','price_acre_sales', 'acres', 'price_acre_list' ], axis=1)
df0 = df0.rename(columns={'city/location':'city_location'}) #, inplace=True)

# Drop these as they influence the target and are irrelevant
df0 = df0.drop(['selling_office_mlsid', 'selling_office_name', 'selling_agent_mlsid', 'selling_agent_full_name'], axis=1)

In [ ]:
# Get a new coutnt of shape
print('The shapes of dataset as we proceed: {}, {}'.format(df_import.shape, df0.shape))

## Fill missing values

In [ ]:
# Fill nulls of categorical data with a place holder instead of removing them so not to reduce the frequency of the target variable
df0['lot_size'].fillna(df0['lot_size'].median(), inplace=True)
df0['area'].fillna("value missing", inplace=True)
df0['street_number'].fillna("value missing", inplace=True)
df0['subdivision'].fillna("value missing", inplace=True)
df0['market_area'].fillna("value missing", inplace=True)
df0['school_elementary'].fillna("value missing", inplace=True)
df0['school_middle'].fillna("value missing", inplace=True)
df0['school_high'].fillna("value missing", inplace=True)
df0['room_count'].fillna(df0['room_count'].median(), inplace=True)
df0['building_sqft'].fillna(df0['building_sqft'].median(), inplace=True)


In [ ]:
# Get a new coutnt of shape
print('The shapes of dataset as we proceed: {}, {}'.format(df_import.shape, df0.shape))

In [ ]:
# Use a dict for min and max normal values for each column. Drop rows with values outside of this range 
value_range = {
    'building_sqft': (800, 5000),
    'lot_size': (3000, 30000),
    'bedrooms': (1,8),
    'baths_full': (1, 5),
    'baths_half': (0, 5),
    'baths_total': (1, 10),
    'room_count': (1, 10),
    'stories': (1,2),
    'no_of_garage_cap': (0,3),
    'dom': (0, 600),
    'cdom': (0, 600)}

def drop_extreme_rows(df):
    for c in range(0, len(list(value_range.keys()))):
        col_name = list(value_range.keys())[c]
        col_min_val = list(value_range.values())[c][0]
        col_max_val = list(value_range.values())[c][1]
        df = df[df[col_name].between(col_min_val, col_max_val)]
    return df
df0 = drop_extreme_rows(df0)

In [ ]:
# Get a new coutnt of shape
print('The shapes of dataset as we proceed: {}, {}'.format(df_import.shape, df0.shape))

In [ ]:
df1 = df0.copy()

## Feature Engineering

In [ ]:
# Add Target
df1['target'] = np.where((df1.status == "expired") | (df1.status == "terminated") | (df1.status == "withdrawn"), 1,0)
# Reorder the columns so those important one are shown on the left side
df1 = df1[ ['target'] + [ col for col in df1.columns if col != 'target' ] ]
# Slice the data into more comparable property types
df1 = df1[df1['property_type']=='single-family']
print('The shapes of dataset as we proceed: {}, {}, {}'.format(df_import.shape, df0.shape, df1.shape))

In [ ]:
# Add a column with ones
df1['ones'] = 1
# Create the ratio of lot size as of building size for filling the nans lalter
df1['lot_build_ratio'] = df1['lot_size'].div(df1['building_sqft'].where(df1['lot_size'] != 0, 0 ))#np.nan))
# Get the median of lot size to building size ratio
lot_build_ratio_median = df1.loc[df1['lot_build_ratio'] !='NaN', 'lot_build_ratio'].median()
df1["lot_build_ratio"] = np.where(lot_build_ratio_median =='NaN', lot_build_ratio_median, lot_build_ratio_median)
# Get a new coutnt of shape
print('The shapes of dataset as we proceed: {}, {}, {}'.format(df_import.shape, df0.shape, df1.shape))

In [ ]:
# Find the median of dates
year_built_median = df1['year_built'].astype('datetime64[ns]').quantile(.5)
list_date_median = df1['list_date'].astype('datetime64[ns]').quantile(.5)
# Fill dates with null
df1['year_built'].fillna(year_built_median, inplace=True)
df1['list_date'].fillna(list_date_median, inplace=True)
# Get a new coutnt of shape
print('The shapes of dataset as we proceed: {}, {}, {}'.format(df_import.shape, df0.shape, df1.shape))

In [ ]:
# df2 and df3 are kept unused on purpose

In [ ]:
df4 = df1.copy()

In [ ]:
# check to see if the unique index is duplicated. A duplication cause problem later
move_to_left_get_dup(df0, 'import_id')

In [ ]:
# Add price boundry
lower_bound = 200000
upper_bound = 750000
df4 = df4[((df4['list_price'] >= lower_bound) & (df4['list_price'] <=upper_bound))]

# Get a new coutnt of shape
print('The shapes of dataset as we proceed: {}, {}, {}'.format(df0.shape, df1.shape, df4.shape))

## Add rank for the target

In [ ]:
# Add performance rank to identify top producers
df4['target_sum'] = df4.groupby('list_office_mlsid')['target'].transform('sum')
df4['target_rank'] = df4['target_sum'].rank(method='dense', ascending=False)

In [ ]:
ax = df4.hist(column='target_rank', bins=10, grid=False, figsize=(10,8), color='#86bf91', zorder=2, rwidth=0.9)

ax = ax[0]
for x in ax:

    # Despine
    x.spines['right'].set_visible(False)
    x.spines['top'].set_visible(False)
    x.spines['left'].set_visible(False)

    # Switch off ticks
    x.tick_params(axis="both", which="both", bottom="off", top="off", labelbottom="on", left="off", right="off", labelleft="on")

    # Draw horizontal axis lines
    vals = x.get_yticks()
    for tick in vals:
        x.axhline(y=tick, linestyle='dashed', alpha=0.4, color='#eeeeee', zorder=1)

    # Remove title
    x.set_title("Target rank of Listing Agent ")

## See Listing Price vs. Size (Sqft)

In [ ]:
# prepare the data for plotting
# separate x and y
x = df4["list_price"]
y = df4["building_sqft"]

# instanciate the figure
fig = plt.figure(figsize = (12, 6))
# in this case we use gridspec.
gs = fig.add_gridspec(5, 5)
ax1 = fig.add_subplot(gs[:4, :-1])

# plot the data
# main axis: scatter plot
ax1.scatter(x, y) #, c = df4.target.astype('category').cat.codes) 

# set the labels for x and y
ax1.set_xlabel("Building, sqf")
ax1.set_ylabel("Listing Price, k$")

# set the title for the main plot
ax1.set_title("Building Size vs Listin Price")

# prettify the plot
# get rid of some of the spines to make the plot nicer
ax1.spines["right"].set_color("None")
ax1.spines["top"].set_color("None")

# using familiar slicing, get the bottom axes and plot
ax2 = fig.add_subplot(gs[4:, :-1])
ax2.hist(x, 40, orientation = 'vertical', color = "pink")

# invert the axis (it looks up side down)
ax2.invert_yaxis()

# prettify the plot
# set the ticks to null
ax2.set_xticks([])
ax2.set_yticks([])
# no axis to make plot nicer
ax2.axison = False

# using familiar slicing, get the left axes and plot
ax3 = fig.add_subplot(gs[:4, -1])
ax3.hist(y, 40, orientation = "horizontal", color = "pink")

# prettify the plot
# set the ticks to null
ax3.set_xticks([])
ax3.set_yticks([])
# no axis to make plot nicer
ax3.axison = False

# make all the figures look nicier
fig.tight_layout()


## Target vs. Building & Lot Size 

In [ ]:
# instanciate the figure
fig = plt.figure(figsize = (12, 6))
ax = fig.add_subplot(1,1,1,)

# iterate over each category and plot the data. This way, every group has it's own color. Otherwise everything would be blue
for cat in sorted(list(df4["target"].unique())):
# filter x and the y for each category
    lot = df4[df4["target"] == cat]["lot_size"]
    build = df4[df4["target"] == cat]["building_sqft"]
    # plot the data
    ax.scatter(lot, build, label = cat, s = 40)
    
# prettify the plot

# eliminate 2/4 spines (lines that make the box/axes) to make it more pleasant
ax.spines["top"].set_color("None") 
ax.spines["right"].set_color("None")

# set a specific label for each axis
ax.set_xlabel("Lot Size, sqf") 
ax.set_ylabel("Building, sqf")

# change the lower limit of the plot, this will allow us to see the legend on the left
ax.set_xlim(-0.01) 
ax.set_title("Lost Size vs. Building Size: Target")
ax.legend(loc = "upper left", fontsize = 10);

## Visual of all numeric features 

In [ ]:
# Get all numerica columns
numerics = ['int_', 'int8', 'uint8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']

df4_num = df4.select_dtypes(include=numerics)
num_cols = df4_num.columns
# Plot all of them
df4[num_cols].plot(subplots=True, layout=(8, 5), figsize=(30, 20), sharex=False);

## Study outliers 

## List Price vs Close Price: Size, Mean, Median, Max and Min

In [ ]:
pivot1 = pd.pivot_table(df4,index=["target"],values=["list_price", "close_price"], aggfunc=[len, np.mean, np.median, np.max, np.min], margins=True).round(0)

# function to pivot showing in 1000$
def format(x):
        return "${:.1f}K".format(x/1000)

# Apply function to specifc columns

lst = pivot1.columns[2:10]
for l in lst:
    pivot1[l] = pivot1[l].apply(format)
pivot1

## Building size vs List Price: Size, Mean, Median, Max and Min

In [ ]:
#pivot_df = df0.loc[df0['property_type'].isin(['single-family', 'townhouse/condo', 'mid/hi-rise condo'])]
pivot2 = pd.pivot_table(df4,index=["target"],values=["building_sqft", "list_price"], aggfunc=[len, np.mean, np.median, np.max, np.min], margins=True).round(0)

# function to pivot showing in 1000$
def format(x):
        return "{:.1f}K".format(x/1000)

# Apply function to specifc columns

lst = pivot2.columns[2:10]
for l in lst:
    pivot2[l] = pivot2[l].apply(format)
pivot2

## Explore and fix any outliers

In [ ]:
# Get all numerica columns
numerics = ['int_', 'int8', 'uint8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']

df4_num = df4.select_dtypes(include=numerics)
col_names = df4_num.columns

# Eliminate unwanted elements
unwanted = {'close_price', 'new_construction', 'pool_private', 'ones', 'target', 'target_sum', 'target_rank'}
col_names = [ele for ele in col_names if ele not in unwanted]
                      
# Get a dict of numerial columns
col_dict = {c: i for i, c in enumerate(col_names, start=1)}

# Detect outliers in each variable using box plots.
plt.figure(figsize=(20,70))

for variable,i in col_dict.items():
                     plt.subplot(30,3,i)
                     plt.boxplot(df4[variable],whis=1.5)
                     plt.title(variable)
plt.show()

In [ ]:
# Calculate number of outliers and its percentage in each variable using Tukey's method.
for variable in col_dict.keys():
    q75, q25 = np.percentile(df4[variable], [75 ,25])
    iqr = q75 - q25
    min_val = q25 - (iqr*1.5)
    max_val = q75 + (iqr*1.5)
    col_length = len(np.where(df4[variable])[0])
    print("Number of outliers and percentage of it in {} : {} and {}".format(variable,
                                                len((np.where((df4[variable] > max_val) | (df4[variable] < min_val))[0])),
                                                len((np.where((df4[variable] > max_val) | (df4[variable] < min_val))[0]))*100/col_length))

In [ ]:
print('The shapes of dataset as we proceed: {}'.format(df4.shape))

In [ ]:
# check to see if the unique index is duplicated. A duplication cause problem later
move_to_left_get_dup(df4, 'import_id')

## Winsorize outliers 

In [ ]:
# Get all numerica columns
numerics = ['int_', 'int8', 'uint8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']

df4_num = df4.select_dtypes(include=numerics)
col_names = df4_num.columns

# Eliminate unwanted elements
unwanted = {'close_price', 'new_construction', 'pool_private', 'ones', 'target', 'target_sum', 'target_rank'}
col_names = [ele for ele in col_names if ele not in unwanted]

# Define Winsorize funtion
def using_mstats(s):
    return winsorize(s, limits=[0.01, 0.01])


# Apply the function
#df4[col_names] = df4[col_names].apply(lambda x: winsorize(x, limits=[0.01, 0.01]))

limit = 0.1
df4['no_of_garage_cap'] = winsorize(df4['no_of_garage_cap'], limits=limit)

In [ ]:
print('The shapes of dataset as we proceed: {}'.format(df4.shape))

In [ ]:
move_to_left_get_dup(df4, 'import_id')

## Copy to new dataframe 

In [ ]:
df5 = df4.copy()
print('The shapes of dataset as we proceed: {}, {}, {}, {}'.format(df0.shape, df1.shape, df4.shape, df5.shape))

## Perform simple regression for predicting the listing price 

In [ ]:
describe_with_nulls(df5)

In [ ]:
# Y is the target variable
Y = df5['list_price']
# X is the feature set which includes
numerics = ['int_', 'int8', 'uint8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
print(numerics)
df5_num = df5.select_dtypes(include=numerics)
print(df5_num.shape)
X = df5_num.drop(['list_price', 'close_price', 'target'], axis=1)


# We create a LinearRegression model object
# from scikit-learn's linear_model module.
lrm = linear_model.LinearRegression()

# fit method estimates the coefficients using OLS
lrm.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', lrm.coef_)
print('\nIntercept: \n', lrm.intercept_)

# Gauss-Markov Theorem

## According to the Gauss–Markov theorem, in a linear regression model the ordinary least squares (OLS) estimator gives the best linear unbiased estimator (BLUE) of the coefficients, provided that

### Target variable should be a linear function of the model's coefficients
### The expectation of errors (residuals) is 0
### The errors have equal variance — homoscedasticity of errors
### Features should have low multicollinearity
### The errors are uncorrelated
### The explanatory variables and errors should be independent (exogeneity)


## Check for linearity

### The way we have specified our model here supports the linearity assumption

## Check assumption that error term should be zero on average¶

In [ ]:
predictions = lrm.predict(X)
errors = Y - predictions

print("Mean of the errors in the target model is: {}".format(np.mean(errors)))

### Since we have a constant term in the model, the average of the model's error is effectively zero.

## Check for the homoscedasticity assumption

## A model is homoscedastic when the distribution of its error terms (known as "scedasticity") is consistent for all predicted values. In other words, the error variance shouldn't systematically change across observations. 

In [ ]:
plt.scatter(predictions, errors)
plt.xlabel('Predicted')
plt.ylabel('Residual')
plt.axhline(y=0)
plt.title('Residual vs. Predicted')
plt.show()

### We can see that our error terms are not consistently distributed. 

### There are several formal statistical tests that we can use to determine whether there is heteroscedasticity in the error terms. Two of these are Barlett and Levene

### Perform Barlett and Levene tests

In [ ]:
from scipy.stats import bartlett
from scipy.stats import levene

bart_stats = bartlett(predictions, errors)
lev_stats = levene(predictions, errors)

print("Bartlett test statistic value is {0:3g} and p value is {1:.3g}".format(bart_stats[0], bart_stats[1]))
print("Levene test statistic value is {0:3g} and p value is {1:.3g}".format(lev_stats[0], lev_stats[1]))

### The p-values of both tests is meaning we can reject the null hypothesis which again means our errors are hetetroscedastic.

## Check for low multicollinearity

### Individual features should be only weakly correlated with one another, and ideally completely uncorrelated. When features are correlated, they may both explain the same pattern of variance in the outcome. The model will attempt to find a solution, potentially by attributing half the explanatory power to one feature and half to the other. This isn’t a problem if our only goal is prediction, because then all that matters is that the variance gets explained. However, if we want to know which features matter most when predicting an outcome, multicollinearity can cause us to underestimate the relationship between features and outcomes.

#### We can simply use correlation matrix of the features to discover any multicollinearity

In [ ]:
corr = X.corr()
corr.style.background_gradient(cmap='coolwarm')
# 'RdBu_r' & 'BrBG' are other good diverging colormaps

## Several coefficients are correlated 

## Check if error terms are uncorrelated as they should be 

In [ ]:
plt.plot(errors)
plt.show()

### Another option is to check the autocorrelation function. 

In [ ]:
from statsmodels.tsa.stattools import acf

acf_data = acf(errors)

plt.plot(acf_data[1:])
plt.show()

### We have high multicollinearity not supporting the OLS assumptions

## Check if the features are correlated with the errors. A high correlation violates the OLS assumption (exogeneity)

### This assumption, is arguably the most important one: explanatory variables and errors should be independent. If this assumption doesn't hold, then the model's predictions will be unreliable as the estimates of the coefficients would be biased. This assumption is known as the exogeneity.

In [ ]:
# Convert errors to series
error_series = pd.Series(errors)
X_copy = X.copy()
X_copy['errors'] = error_series
corr = X_copy.corr()
corr.style.background_gradient(cmap='BuPu')
# 'RdBu_r' & 'BrBG' are other good diverging colormaps

## Normality of residuals

### When this assumption is violated, it causes problems with calculating confidence intervals and various significance tests for coefficients. When the error distribution significantly departs from Gaussian, confidence intervals may be too wide or too narrow. Although it is not an assumption of OLS, it still can impact our results. Specifically, normality of errors is not required to apply OLS to a linear regression model, but in order to measure the statistical significance of our estimated coefficients, error terms must be normally distributed. 

###  An informal way of doing this is by visualizing the errors in a QQ plot or to look at the histogram:

In [ ]:
rand_nums = np.random.normal(np.mean(errors), np.std(errors), len(errors))

plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.scatter(np.sort(rand_nums), np.sort(errors)) # we sort the arrays
plt.xlabel("the normally distributed random variable")
plt.ylabel("errors of the model")
plt.title("QQ plot")

plt.subplot(1,2,2)
plt.hist(errors)
plt.xlabel("errors")
plt.title("Histogram of the errors")

plt.tight_layout()
plt.show()

### Above shows that our erros are almost normally distributed.

### Perform Jargue Berra and normals tests

In [ ]:
from scipy.stats import jarque_bera
from scipy.stats import normaltest

In [ ]:
jb_stats = jarque_bera(errors)
norm_stats = normaltest(errors)

print("Jarque-Bera test statistics is {0} and p value is {1}".format(jb_stats[0], jb_stats[1]))
print("Normality test statistics is {0} and p value is {1}".format(norm_stats[0], norm_stats[1]))

### The p-values of both tests are zero indicating that our errors are normally distributed.

In [ ]:
# Y is the target variable
Y = df5['list_price']
# X is the feature set which includes
numerics = ['int_', 'int8', 'uint8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
print(numerics)
df5_num = df5.select_dtypes(include=numerics)
print(df5_num.shape)
X = df5_num.drop(['list_price', 'close_price', 'target'], axis=1)
X = sm.add_constant(X)
# We fit an OLS model using statsmodels
results = sm.OLS(Y, X).fit()
print(results.summary())

### Most of the p-values of coefficients are close to zero indicating they are statistically significants. 

## Copy dataframe 

In [ ]:
df6 = df5.copy()
print('The shapes of dataset as we proceed: {}, {}, {}, {}, {}'.format(df0.shape, df1.shape, df4.shape, df5.shape, df6.shape))

## Add dummy variables for some categorical data 

In [ ]:
	cat_lst = [
    #
    #'street_name','city_location', 'style'\
    #'subdivision'
    #, 'market_area'\
    #,'school_district', 'school_elementary', 'school_middle', 'school_high', 
    #,'list_agent_mlsid' \
    #,'list_office_mlsid'\
    #'selling_office_mlsid', 'close_date', 'year_built' \
    #'zip_code'
    #'area'
    'county'
    ]		

In [ ]:
dummies = pd.get_dummies(data= df6, columns=cat_lst)
df7 = dummies.copy()
print('The shapes of dataset as we proceed: {}, {}, {}, {}, {}, {}'.format(df0.shape, df1.shape, df4.shape, df5.shape, df6.shape, df7.shape))

In [ ]:
#df7[df7.duplicated('import_id')]

In [ ]:
move_to_left_get_dup(df7, 'import_id')

In [ ]:
describe(df6)

In [ ]:
print('The shapes of dataset as we proceed: {}, {}, {}, {}, {}, {}'.format(df0.shape, df1.shape, df4.shape, df5.shape, df6.shape, df7.shape))

In [ ]:
pivot2 = pd.pivot_table(df7,index=["target","zip_code"],aggfunc=[len], margins=True).round(0)

#pivot2.columns
#pivot2.sort_values(by=('len', 'target'), ascending=False)
#pivot2

In [ ]:
# Y is the target variable
Y = df7['list_price']

# X is the feature set which includes
numerics = ['int_', 'int8', 'uint8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df7_num = df7.select_dtypes(include=numerics)
X = df7_num.drop(['list_price', 'close_price', 'target'], axis=1)

In [ ]:
#new_X = pd.DataFrame(mca.transform(X), columns=['mca%i' % i for i in range(n_components)], index=df7.index)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 465)
print("df7.shape:{}, X.shape:{}, Y.shape:{}, X_train.shape:{}, X_test.shape:{}, y_train.shape:{}, y_test.shape{}".format(df7.shape, X.shape, Y.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape))

## Ridge regression  

In [ ]:
alphas = np.array([1000000, 1000, 10,  1, 0.1, 0.01, 0.001])
model = Ridge(fit_intercept = True)
grid = GridSearchCV(estimator=model, cv = 5, param_grid=dict(alpha=alphas))
grid.fit(X_train, y_train)
print(grid)
print(grid.best_score_)
print(grid.best_estimator_.alpha)

In [ ]:
# Run the Ridge
regression = Ridge(alpha=1)
regression.fit(X_train, y_train)
y_pred = regression.predict(X_test)

In [ ]:
# Print the regression result performance
print("--------------Ridge--------------------")
print("------- Number of Observation ---------")
print("The number of observations in training set is {}".format(X_train.shape[0]))
print("The number of observations in test set is {}".format(X_test.shape[0]))
print("----------------------------------------")
print("----- Performance on training data -----")
regression.fit(X_train, y_train)
y_pred = regression.predict(X_train)
regression_results(y_train, y_pred)
print("----------------------------------------")
print("----- Performance on test data -----")
regression.fit(X_train, y_train)
y_pred = regression.predict(X_test)
regression_results(y_test, y_pred)
print("----------------------------------------")
print("----- Coefficients and intercept data -----")
print(regression.coef_)
#print(regression.intercept_)

## Lasso regression  

In [ ]:
alphas = np.array([1000000, 100000, 1000, 1, 0.1, 0.01, 0.001])
model = Lasso(fit_intercept = True)
grid = GridSearchCV(estimator=model, cv=5, param_grid=dict(alpha=alphas))
grid.fit(X_train, y_train)
print(grid)
print(grid.best_score_)
print(grid.best_estimator_.alpha)

In [ ]:
# Run the Lasso
regression = Lasso(alpha=1)
regression.fit(X_train, y_train)
y_pred = regression.predict(X_test)

In [ ]:
# Print the regression result performance
print("-------------Lasso----------------------")
print("------- Number of Observation ---------")
print("The number of observations in training set is {}".format(X_train.shape[0]))
print("The number of observations in test set is {}".format(X_test.shape[0]))
print("----------------------------------------")
print("----- Performance on training data -----")
regression.fit(X_train, y_train)
y_pred = regression.predict(X_train)
regression_results(y_train, y_pred)
print("----------------------------------------")
print("----- Performance on test data -----")
regression.fit(X_train, y_train)
y_pred = regression.predict(X_test)
regression_results(y_test, y_pred)
print("----------------------------------------")
print("----- Coefficients and intercept data -----")
print(regression.coef_)
print(regression.intercept_)

## ElasticNet regression  

In [ ]:
#alphas = np.array([1000000, 100000, 1000, 1, 0.1, 0.01, 0.001])
param_grid = {"max_iter": [1, 5, 10], "alpha":[1000000, 100000, 1000, 1, 0.1, 0.01, 0.001], "l1_ratio": np.arange(0.0, 1.0, 0.1)}

model = ElasticNet(fit_intercept = True)
grid = GridSearchCV(estimator=model, cv = 5, param_grid=param_grid)#) #dict(alpha=alphas))
grid.fit(X_train, y_train)
print(grid)
print(grid.best_score_)
print(grid.best_estimator_.max_iter)
print(grid.best_estimator_.alpha)
print(grid.best_estimator_.l1_ratio)

In [ ]:
# Run the ElasticNet
regression = ElasticNet(max_iter=5, alpha=0.001, l1_ratio=0.7)
regression.fit(X_train, y_train)
y_pred = regression.predict(X_test)

In [ ]:
# Print the regression result performance
print("-----------ElasticNet------------------")
print("------- Number of Observation ---------")
print("The number of observations in training set is {}".format(X_train.shape[0]))
print("The number of observations in test set is {}".format(X_test.shape[0]))
print("----------------------------------------")
print("----- Performance on training data -----")
regression.fit(X_train, y_train)
y_pred = regression.predict(X_train)
regression_results(y_train, y_pred)
print("----------------------------------------")
print("----- Performance on test data -----")
regression.fit(X_train, y_train)
y_pred = regression.predict(X_test)
regression_results(y_test, y_pred)
print("----------------------------------------")
print("----- Coefficients and intercept data -----")
#print(regression.coef_)
#print(regression.intercept_)

## Random Forest Regressor

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 2)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 50, num = 2)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rfr = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
rf_random = RandomizedSearchCV(estimator = rfr, param_distributions = random_grid, n_iter = 1, cv = 3, verbose=2, random_state=42, n_jobs = 8 )#-1 means all cores are used)
#Fit the random search model
rf_random.fit(X_train, y_train)

In [ ]:
rf_random.best_params_

In [ ]:
# To determine if random search yielded a better model, we compare the base model with the best random search model.
def evaluate(model, X_test, y_test):
    predictions = model.predict(X_test)
    print(predictions.shape)
    y_test = y_test.values
    errors = abs(predictions - y_test)
    mape = 100 * np.mean(errors / y_test)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)

In [ ]:
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test, y_test)

In [ ]:
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

## Random search allowed us to narrow down the range for each hyperparameter. Now that we know where to concentrate our search, we can explicitly specify every combination of settings to try. 

In [ ]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 100, 120],
    'max_features': ['auto'],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [2, 4, 8],
    'n_estimators': [1200, 1400, 1600]
}
# Create a based model
rfr = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rfr, param_grid = param_grid, 
                          cv = 3, n_jobs = 5, verbose = 2)
# Fit the grid search to the data
grid_search.fit(X_train, y_train)
grid_search.best_params_

In [ ]:
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, X_test, y_test)

In [ ]:
print('Improvement of {:0.2f}%.'.format( 100 * (grid_accuracy - base_accuracy) / base_accuracy))

## It seems we have about maxed out performance

## All the models perform poorly. This require additional feature engineering 

## Pick one of fited model for market price prediction

In [ ]:
# Run the Ridge
ridge_regression = Ridge(alpha=1)
ridge_regression.fit(X_train, y_train)
y_pred = ridge_regression.predict(X)

# Create temporary pandas to combine (left join) original dataset with new market price prediction
X1 = X.copy()
X1['value_predicted'] = y_pred
X1.drop(X1.columns.difference(['import_id','value_predicted']), 1, inplace=True)
df8 = pd.merge(left=df7, right=X1, how='left', left_on='import_id', right_on='import_id')
print("df7.shape: {}, df8.shape: {}, y_pred.shape:{}, X_train.shape:{}, y_train.shape:{}".format(df7.shape, df8.shape, y_pred.shape, X_train.shape, y_train.shape))
del X1

In [ ]:
move_to_left_get_dup(df8, 'import_id')

In [ ]:
def get_distict_count(df, col_name):
    #df_empty = pd.DataFrame({'A' : []})
    df_empty = df.groupby([col_name]).size()
    return df_empty

In [ ]:
 # Calculate how much listing price deviates from the expected value

df8['value_price_delta'] = df8['value_predicted'] - df8['list_price']
df8['value_price_delta_percent'] = (df8['value_price_delta']/df8['list_price'])*100 
df8['value_price_delta_percent'] = df8['value_price_delta_percent']#a.astype(int)

In [ ]:
numerics = ['int_', 'int8', 'uint8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df8_num = df8.select_dtypes(include=numerics)
df8_num.columns
Y = df8['target']
X = df8_num.drop(['target', 'close_price'], axis=1)

## Check the balance of the target features

In [ ]:
df8.shape, X.shape, y_train.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.35, random_state = 465)
print("X.shape:{}, Y.shape:{}, X_train.shape:{}, X_test.shape:{}, y_train.shape:{}, y_test.shape{}".format(X.shape, Y.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape))

In [ ]:
describe_with_nulls(df8)

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

#iris = datasets.load_iris()

svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("linear_svc", LinearSVC(C=1, loss="hinge")),
    ])

svm_clf.fit(X_train, y_train)

In [ ]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

pca = PCA(n_components=None, svd_solver='randomized', whiten=True, random_state=42)
svc = SVC(kernel='rbf', class_weight='balanced')
model = make_pipeline(pca, svc)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.35, random_state = 465)


In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'svc__C': [1, 5, 10, 50],
              'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid)

%time grid.fit(X_train, y_train)
print(grid.best_params_)

In [ ]:
model = grid.best_estimator_
yfit = model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, yfit))#)                           

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

mat = confusion_matrix(y_test, yfit)
sns.heatmap(mat.T, square=True, annot=True, fmt='d',
            cmap='Blues', cbar=False,
            xticklabels= df8.target,
            yticklabels= df8.target)
plt.xlabel('true label')
plt.ylabel('predicted label')

In [ ]:
d